In [1]:
import keras
from keras.models import Sequential
from keras.layers import Dense
import pandas as pd
import numpy as np
from sklearn import ensemble,metrics,cross_validation

Using TensorFlow backend.
C:\Users\sharm_000\Anaconda2\envs\py36\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
data=pd.read_csv('train.csv')
test_=pd.read_csv('test.csv')

In [3]:
data['val']='train'
test_['val']='test'

In [4]:
all_data=pd.concat([data,test_])

In [5]:
def gini(list_of_values):
    sorted_list = sorted(list(list_of_values))
    height, area = 0, 0
    for value in sorted_list:
        height += value
        area += height - value / 2.
    fair_area = height * len(list_of_values) / 2
    return (fair_area - area) / fair_area
  
def normalized_gini(y_pred, y):
    normalized_gini = gini(y_pred)/gini(y)
    return normalized_gini

def gini_xgb(preds, dtrain):
    labels = dtrain.get_label()
    gini_score = gini_normalized(labels, preds)
    return [('gini', gini_score)]

In [6]:
cat_cols=[i for i in all_data.columns if 'cat' in i]

In [7]:
new_data=pd.get_dummies(all_data,columns=cat_cols,drop_first=True)

In [8]:
new_data.shape

(1488028, 216)

In [9]:
X=[i for i in new_data.columns if i not in ['id','target','val']]
Y=['target']

In [10]:
train_data=new_data[new_data['val']=='train']
test_data=new_data[new_data['val']=='test']

In [11]:
del all_data
del data
del new_data

In [12]:
train_X,test_X,train_y,test_y=cross_validation.train_test_split(train_data[X],train_data[Y],test_size=.3,stratify=train_data[Y])
train_X.shape,train_y.shape,test_X.shape,test_y.shape

((416648, 213), (416648, 1), (178564, 213), (178564, 1))

In [13]:
input_dim=train_X.shape[1]
input_dim

213

In [18]:
model = Sequential()
model.add(Dense(100, activation='relu', input_dim=input_dim))
model.add(Dense(100, activation='relu', input_dim=100))
model.add(Dense(100, activation='relu', input_dim=100))
model.add(Dense(50, activation='relu', input_dim=100))
model.add(Dense(1, activation='sigmoid', input_dim=50))
model.compile(optimizer='rmsprop', loss='binary_crossentropy')

In [19]:
model.fit(train_X.values, train_y.values, epochs=10)

Epoch 1/10
416648/416648 [==============================] - 23s - loss: 0.2694    
Epoch 2/10
416648/416648 [==============================] - 21s - loss: 0.5875    
Epoch 3/10
416648/416648 [==============================] - 21s - loss: 0.5875    
Epoch 4/10
416648/416648 [==============================] - 20s - loss: 0.5875    
Epoch 5/10
416648/416648 [==============================] - 21s - loss: 0.5875    
Epoch 6/10
416648/416648 [==============================] - 21s - loss: 0.5875    
Epoch 7/10
416648/416648 [==============================] - 20s - loss: 0.5875    
Epoch 8/10
416648/416648 [==============================] - 21s - loss: 0.5875    
Epoch 9/10
416648/416648 [==============================] - 21s - loss: 0.5875    
Epoch 10/10
416648/416648 [==============================] - 20s - loss: 0.5875    


In [16]:
pred=np.ravel(model.predict_proba(test_X.values))

177888/178564 [============================>.] - ETA: 0s

In [17]:
gini(list(pred)),gini(test_y.values),normalized_gini(list(pred),test_y.values)

(0.80796386752398752, array([ 0.96355368]), array([ 0.83852502]))

In [ ]:
final_pred=model.predict_proba(test_data[X].values)

In [ ]:
final_pred=np.ravel(final_pred)

In [ ]:
output = pd.DataFrame({'id': test_['id'], 'target': final_pred})
output.to_csv("keras_pred3.csv", index=False)   

In [ ]:
K = 5
kf = cross_validation.KFold(train_data.shape[0],n_folds= K, random_state = 3228, shuffle = True)

In [ ]:
xgb_preds=[]
for train_index, test_index in kf:
    print ('A')
    train_X, valid_X = train_data[X].loc[train_index], train_data[X].loc[test_index]
    train_y, valid_y = train_data[Y].loc[train_index], train_data[Y].loc[test_index]

    # params configuration also from anokas' kernel
    xgb_params = {
        'eta': 0.02,
        'max_depth': 6,
        'subsample': 0.9,
        'objective': 'binary:logistic',
        'silent': 1,
        'colsample_bytree': 0.9
    }

    d_train = xgb.DMatrix(train_X, train_y)
    d_valid = xgb.DMatrix(valid_X, valid_y)
    d_test = xgb.DMatrix(test_data[X])
    
    model = xgb.train(xgb_params, d_train, num_boost_round = 400)
                        
    xgb_pred = model.predict(d_test)
    xgb_preds.append(list(xgb_pred))

In [ ]:
preds=[]
for i in range(len(xgb_preds[0])):
    sum=0
    for j in range(K):
        sum+=xgb_preds[j][i]
    preds.append(sum / K)

In [ ]:
output = pd.DataFrame({'id': test_['id'], 'target': preds})
output.to_csv("sol_sub.csv".format(K), index=False)   